In [25]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [38]:
from src.config import settings
from pathlib import Path
import pandas as pd

In [27]:
# See: https://github.com/michalc/sqlite-s3-query

from sqlite_s3_query import sqlite_s3_query

In [28]:
print([item for item in settings if "DOCKER" in item])
    

[]


In [29]:
def get_scaleway_credentials(_):

    # REGION_NAME = settings.REGION_NAME
    # ENDPOINT_URL = settings.ENDPOINT_URL

    SECRETS_FILE = ".secrets.toml"

    if (
        Path(SECRETS_FILE).exists() is True
        or Path(f'../{SECRETS_FILE}').exists() is True  # if being run from notebooks subdirectory
    ):    # if there is a local secrets file
        SECRET_ACCESS_KEY = settings.SCALEWAY_SECRET_ACCESS_KEY
    else:                                      # use environment variables (deployed on Streamlit Sharing)
        SECRET_ACCESS_KEY = os.environ["SCALEWAY_SECRET_ACCESS_KEY"]


    return (
        settings.REGION_NAME,
        settings.ACCESS_KEY_ID,
        SECRET_ACCESS_KEY,
        None
    )

In [30]:
# get_scaleway_credentials(_)

In [31]:
bucket_name = "databooth-beach-swim"
region_name = settings.REGION_NAME
database_name = "daily_beach_data_db.sqlite"

SQLITE_DB_URL = f"https://{bucket_name}.s3.{region_name}.scw.cloud/{database_name}"

In [32]:
SQLITE_DB_URL

'https://databooth-beach-swim.s3.fr-par.scw.cloud/daily_beach_data_db.sqlite'

In [39]:
eg_query = 'SELECT "Beach name" FROM beaches WHERE Region = "SydneyHarbourBeaches"'

eg_query = eg_query.replace("SydneyHarbourBeaches", "?")


In [40]:
eg_query

'SELECT "Beach name" FROM beaches WHERE Region = "?"'

In [42]:
eg_query = 'SELECT "Beach name" FROM beaches WHERE Region = ?'
params=("SydneyHarbourBeaches", )

In [43]:
eg_query

'SELECT "Beach name" FROM beaches WHERE Region = ?'

In [54]:
def query_return_dataframe(query_str, params, db_url):
    with sqlite_s3_query(url=SQLITE_DB_URL, get_credentials=get_scaleway_credentials) as query:
        with query(query_str, params=params) as (columns, rows):
            df = pd.DataFrame(data=rows, columns=columns)
        return df

In [55]:
df = query_return_dataframe(eg_query, params, SQLITE_DB_URL)

In [56]:
df

,Beach name
